# TNE 2: Principal Component Analysis

The purpose of this tutorial is to use Principal Component Analysis (PCA) 
for dimension reduction applied to images.


In [ ]:
from pylab import *

#import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import pandas as pd
from numpy import linalg as la
from numpy.testing import assert_array_almost_equal

## 1. Application: handwritten digits recognition 5 & 6
We load 2 matrices which contain each a sequence of examples of 16x16 images of handwritten digits which are 5 and 6 here. Each line of the matrix contains 256 pixel values coding for the gray level of a 16x16 image.

In [ ]:
train_5 = np.loadtxt('train_5.txt',delimiter=',')   # 556 samples
train_6 = np.loadtxt('train_6.txt',delimiter=',')   # 664 samples

#### Examples of images:

In [ ]:
# Digit 5
n=7
I = np.reshape(train_5[n,:],(16,16))

plt.imshow(I,cmap='gray')
plt.show()

In [ ]:
# Digit 6
n=2
I = reshape(train_6[n,:],(16,16))

plt.imshow(I,cmap='gray')
plt.show()

#### Separating the training and test sets

We keep in the training set the 145 first images of 5s and the 200 first
images of 6s:

In [ ]:
x_train_brut = np.vstack((train_5[:145,:], train_6[:200,:]))
N_train = np.size(x_train_brut,axis=0)
class_train = np.ones((345,1))   # label 1 for digit 6
class_train[:145] = 0       # label 0 for digit 5

x_test_brut = np.vstack((train_5[145:,:], train_6[200:,:]))
N_test = np.size(train_5,axis=0)+np.size(train_6,axis=0)-N_train

## 2. Principal Component Analysis

The purpose of this part is to observe the respective contributions of
each component of a PCA of images of 5. The function `sklearn.decomposition.PCA` of `scikit-learn` is available. In practice, one must first estimate the mean vector and then work with centered data. 

### Documentation
First have a look at
http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
    

In [ ]:
# Principal component analysis
moy_train = x_train_brut.mean(axis=0)  # all the data, 5 & 6
x_train_centre = x_train_brut-np.tile(moy_train,(N_train,1))

# PCA from scikit-learn
from sklearn.decomposition import PCA
n_components = 250
pca = PCA(n_components=n_components)
pca.fit(x_train_centre)  # you may forget centering that is done by sklearn PCA

singval = pca.singular_values_   # eigenvalues
comp = pca.components_           # principal components
proj = pca.transform(x_train_centre)  # computes the projection coefficients

### Display the averaged images of 5 & 6 respectively

In [ ]:
I_moy = np.reshape(moy_train,(16,16))   # averaged image = mean 
plt.imshow(I_moy,cmap = 'gray')
plt.show()

### Display an example rebuilt from the 1st component only

In [ ]:
n=7   # choice of image no n=7 or any other
I = I_moy + proj[n,0]*np.reshape(comp[0,:],(16,16)) # adding the 1st PCA component
plt.imshow(I,cmap='gray')
plt.show()

### Exercise 1: PCA & approximation

1. Read the documentation of function `PCA` and identify the input and output parameters.
2. Implement a progressive reconstruction of an image of digit 5 by adding the successive 
contribution of principal components.
3. Observe graphical results. How many components are necessary to obtain a 
reconstruction that you may consider as acceptable? nice? very nice?
4. Optional question: do the same for 6.


<u>Answer 1</u> : Description of the PCA function :
- Inputs parameters:    
  - n_components = Number of components to keep. Defaults to all components
  - copy = If False, data passed to fit are overwritten and running fit(X).transform(X) will not yield the expected results, use fit_transform(X) instead
  - whiten = the components_ vectors are multiplied by the square root of n_samples and then divided by the singular values to ensure uncorrelated outputs with unit component-wise variances.
  - svd_solver = Solver in {'auto', 'full', 'arpack', 'randomized'}
  - tol = Tolerance for singular values computed by svd_solver == ‘arpack’. Must be of range [0.0, infinity[
  - iterated_power = Number of iterations for the power method computed by svd_solver == ‘randomized’. Must be of range [0, infinity[.
  - n_oversamples = This parameter is only relevant when svd_solver="randomized". It corresponds to the additional number of random vectors to sample the range of X so as to ensure proper conditioning
  - power_iteration_normalizer = Power iteration normalizer for randomized SVD solver. Not used by ARPACK. See randomized_svd for more details.
  - random_state = Used when the ‘arpack’ or ‘randomized’ solvers are used. Pass an int for reproducible results across multiple function calls
- Output attributes :
  - components_ = Principal axes in feature space, representing the directions of maximum variance in the data. Equivalently, the right singular vectors of the centered input data, parallel to its eigenvectors. The components are sorted by explained_variance_
  - explained_variance_ = The amount of variance explained by each of the selected components. The variance estimation uses n_samples - 1 degrees of freedom. Equal to n_components largest eigenvalues of the covariance matrix of X
  - explained_variance_ratio_ = Percentage of variance explained by each of the selected components. If n_components is not set then all components are stored and the sum of the ratios is equal to 1.0.
  - singular_values_ = The singular values corresponding to each of the selected components. The singular values are equal to the 2-norms of the n_components variables in the lower-dimensional space
  - mean_ = Per-feature empirical mean, estimated from the training set. Equal to X.mean(axis=0).
  - n_components_ = The estimated number of components. When n_components is set to ‘mle’ or a number between 0 and 1 (with svd_solver == ‘full’) this number is estimated from input data. Otherwise it equals the parameter n_components, or the lesser value of n_features and n_samples if n_components is None
  - n_features_ = Number of features in the training data
  - n_samples_ = Number of samples in the training data
  - noise_variance_ = The estimated noise covariance following the Probabilistic PCA model from Tipping and Bishop 1999
  - n_features_in_ = Number of features seen during fit
  - feature_names_in_ = Names of features seen during fit. Defined only when X has feature names that are all strings

<u>Answer 2</u> :

In [ ]:
def progressive_reconstruction(n, proj, comp):
    """Progressive reconstruction of an image by adding successive contributions of the PCA

    Args:
        n (int): Input image
        
    Returns:
        np.ndarray: image reconstructed 
    """
    I = I_moy + np.reshape(proj[n,:] @ comp,(16,16))
    return I

<u>Answer 3</u> : Graphically, we can say that the reconstruction is :
- ACCEPTABLE : 20 to 50 components
- NICE : 50 to 150
- VERY NICE : 150 to All components

In [ ]:
n = 7
n_components = [5,10,20,30,50,100,150,200,250]
fig, ax = plt.subplot_mosaic([[0,1,2],
                              [3,4,5],
                              [6,7,8]], figsize=(16,16))

for inc in range(len(n_components)):
    pca = PCA(n_components = n_components[inc])
    pca.fit(x_train_centre)
    comp = pca.components_
    proj = pca.transform(x_train_centre)
    
    I_reconstructed = progressive_reconstruction(n, proj, comp)
    ax[inc].imshow(I_reconstructed, cmap="gray")
    ax[inc].set_title(f"Reconstruction with {n_components[inc]} components")
plt.show()

<u>Answer 4</u> : For the 6 images, we can accept the following results :
- ACCEPTABLE : 10 to 50 components
- NICE : 50 to 150
- VERY NICE : 150 to All components

In [ ]:
n = 200
n_components = [5,10,20,30,50,100,150,200,250]
fig, ax = plt.subplot_mosaic([[0,1,2],
                              [3,4,5],
                              [6,7,8]], figsize=(16,16))

for inc in range(len(n_components)):
    pca = PCA(n_components = n_components[inc])
    pca.fit(x_train_centre)
    comp = pca.components_
    proj = pca.transform(x_train_centre)
    
    I_reconstructed = progressive_reconstruction(n, proj, comp)
    ax[inc].imshow(I_reconstructed, cmap="gray")
    ax[inc].set_title(f"Reconstruction with {n_components[inc]} components")
plt.show()

### Exercise 2: PCA & classification
1. Use `proj[0:2,:]` as the coordinates of a point representing each sample
of the training set in a plane. Display the cloud of points associated to
digits 5 and 6 by using 2 different colors.
2. Comment on the repartition of points in the plane. 
3. Do you see how this PCA step makes possible the use of a much simpler classification? 
What would you propose as an alternative to logistic regression of TP3 then?


<u>Answer 1</u> :

In [ ]:
x_5 = [proj[i,0] for i in range(145)]
x_6 = [proj[i,0] for i in range(145,345)]
y_5 = [proj[i,1] for i in range(145)]
y_6 = [proj[i,1] for i in range(145,345)]
plt.scatter(x_5, y_5, label="Images 5")
plt.scatter(x_6, y_6, label="Images 6")
plt.legend()
plt.xlabel("First component")
plt.ylabel("Second component")
plt.title("PCA of Images dataset")
plt.show()

<u>Answer 2</u> : We can clearly identify two clusters for images 5 and 6 only with the plotting along the 2 first principal components. Quantitatively, we can compute the explained variance for the two first components :

In [ ]:
print(f"First component : {pca.explained_variance_ratio_[0]}")
print(f"Second component : {pca.explained_variance_ratio_[1]}")

<u>Answer 3</u> : PCA is a dimension reduction tool, not a classifier. In Scikit-Learn, all classifiers and estimators have a predict method which PCA does not. Once need to fit a classifier on the PCA-transformed data. Scikit-Learn has many classifiers such as the LDA and QDA seen in TP3.
An alternative to TP3 would be to use PCA-transformed data with any classifier in sklearn and there are many :
- Nearest Neighbors
- Linear SVM
- RBF SVM
- Gaussian Process
- Decision Tree
- Random Forest
- Neural Net
- AdaBoost
- Naive Bayes
- QDA/LDA

QDA/LDA :

In [ ]:
class LDA():
    """This class implement the linear discriminant analysis specifically for the loaded dataset synth. This class need to be loaded with a train and test dataset.
    """
    def __init__(self, train, test):
        self.train_df = pd.DataFrame(train,columns = ['classe', 'x1', 'x2'])
        self.test_df = pd.DataFrame(test,columns = ['classe', 'x1', 'x2'])
        self.type = "LDA"
        
        
    def get_pi_estimators(self)->list:
        """Returns the pi estimators for each class.

        Returns:
            list: list of floats
        """
        return [pi for pi in self.train_df.classe.value_counts(normalize=True, ascending=True).values]
    
    def get_mu_estimators(self)->np.ndarray:
        """Returns the mu estimators for each class.

        Returns:
            np.ndarray: an array where each line returns the vector mu (estimator) for the concerned class
        """
        classes = [1,2]
        mu = np.zeros((len(classes), 2))
        for i, c in enumerate(classes):
            mu[i] = self.train_df[self.train_df.classe==c][['x1', 'x2']].sum(axis=0).to_numpy() / self.train_df[self.train_df.classe==c].shape[0]
        return mu
    
    def get_sigma_estimators(self)->np.ndarray:
        """Returns the average sigma estimator.

        Returns:
            np.ndarray: Sigma array in dimension 2x2
        """
        mu = self.get_mu_estimators()
        classes = [1,2]
        sigma_moy = np.zeros((2,2))
        for i, c in enumerate(classes):
            train_df_c = self.train_df[self.train_df.classe==c]
            sigma = np.zeros((2,2))
            for j in range(train_df_c.shape[0]):
                xn = train_df_c[['x1', 'x2']].iloc[j].to_numpy().reshape((2,1))
                sigma += ( train_df_c[['x1', 'x2']].iloc[j].to_numpy().reshape((2,1)) - mu[i].reshape((2,1)) ) @ ( train_df_c[['x1', 'x2']].iloc[j].to_numpy().reshape((2,1)) - mu[i].reshape((2,1)) ).T
            sigma_moy += sigma
        
        return sigma_moy/self.train_df.shape[0]
    
    def get_log_probabilities(self, df:pd.DataFrame)->np.ndarray:
        """Compute the log_probability for the entries.

        Args:
            df (pd.DataFrame): DataFrame with the columns x1 and x2

        Returns:
            np.ndarray: Matrix in dim Nx2 where N is the shape of entries
        """
        pi = self.get_pi_estimators()
        mu = self.get_mu_estimators()
        sigma = self.get_sigma_estimators()
        prediction = np.zeros((len(df), mu.shape[0]))
        for i in range(df.shape[0]):
            x = df[['x1', 'x2']].iloc[i].to_numpy().reshape((2,1))
            y = np.zeros(mu.shape[0])
            for j in range(mu.shape[0]):
                y[j] = np.log(pi[j]) + x.T @ la.inv(sigma) @ mu[j].reshape((2,1)) - 1/2 * mu[j].reshape((2,1)).T @ la.inv(sigma) @ mu[j].reshape((2,1))
            prediction[i] = y
        return prediction
    
    def classification(self, train=True)->np.ndarray:
        """Returns the classification using discriminant analysis.

        Args:
            train (bool, optional): Use the trainset if True and the testset if not. Defaults to True.

        Returns:
            np.ndarray: Vector with class id for each entry.
        """
        if train:
            df = self.train_df
        else:
            df = self.test_df
        prediction = self.get_log_probabilities(df)
        return np.argmax(prediction, axis=1) + 1
    
    def error_rate(self, train=True)->float:
        classes = self.classification(train=train)
        if train:
            results = (classes == self.train_df.classe.to_numpy())
            error_rate = 1 - np.count_nonzero(results)/self.train_df.shape[0]
        else:
            results = (classes == self.test_df.classe.to_numpy())
            error_rate = 1 - np.count_nonzero(results)/self.test_df.shape[0]
        return error_rate
        
    
    def plot_decision_boundary(self):
        """Plot the decision boundary
        """
        Nx1=100 # number of samples for display
        Nx2=100
        x1=np.linspace(-6,6,Nx1)  # sampling of the x1 axis 
        x2=np.linspace(-8,8,Nx2)  # sampling of the x2 axis
        [X1,X2]=np.meshgrid(x1,x2)  
        df = pd.DataFrame({'x1': X1.flatten('F'), 'x2': X2.flatten('F')})
        prediction = self.get_log_probabilities(df)
        classe = list(np.argmax(prediction, axis=1) + 1)
        df['classe'] = [f'classe {i}' for i in classe]
        fig = px.scatter(df, x="x1", y="x2", color = "classe", title=f"Decision boundary with {self.type}")
        fig.write_image(f"figures/decision_boundary_with_{self.type}.png")
        fig.show()
        

In [ ]:
class QDA(LDA):
    """This class implement the linear discriminant analysis specifically for the loaded dataset synth. This class need to be loaded with a train and test dataset.
    """
    def __init__(self, train, test):
        super().__init__(train, test)
        self.type = "QDA"
    
    def get_sigma_estimators(self)->list:
        """Returns the sigma estimator for each class.

        Returns:
            list: List of matrix in dim 2x2
        """
        mu = self.get_mu_estimators()
        classes = [1,2]
        sigma_list =[]
        for i, c in enumerate(classes):
            train_df_c = self.train_df[self.train_df.classe==c]
            sigma = np.zeros((2,2))
            for j in range(train_df_c.shape[0]):
                xn = train_df_c[['x1', 'x2']].iloc[j].to_numpy().reshape((2,1))
                sigma += ( train_df_c[['x1', 'x2']].iloc[j].to_numpy().reshape((2,1)) - mu[i].reshape((2,1)) ) @ ( train_df_c[['x1', 'x2']].iloc[j].to_numpy().reshape((2,1)) - mu[i].reshape((2,1)) ).T
            sigma_list.append(sigma/train_df_c.shape[0])
        
        return sigma_list
    
    def get_log_probabilities(self, df):
        """Compute the log_probability for the entries.

        Args:
            df (pd.DataFrame): DataFrame with the columns x1 and x2

        Returns:
            np.ndarray: Matrix in dim Nx2 where N is the shape of entries
        """
        pi = self.get_pi_estimators()
        mu = self.get_mu_estimators()
        sigma = self.get_sigma_estimators()
        prediction = np.zeros((len(df), mu.shape[0]))
        for i in range(df.shape[0]):
            x = df[['x1', 'x2']].iloc[i].to_numpy().reshape((2,1))
            y = np.zeros(mu.shape[0])
            for j in range(mu.shape[0]):
                y[j] = np.log(pi[j]) - 1/2 * np.log(la.det(sigma[j])) -1/2 *  (x - mu[j].reshape((2,1))).T @ la.inv(sigma[j]) @ (x - mu[j].reshape((2,1)))
            prediction[i] = y
        return prediction

In [ ]:
y_train = np.array([1 for _ in range(145)] + [2 for _ in range(200)]).reshape(-1,1)
proj_train = np.c_[y_train,proj[:, 0:2]]

In [ ]:
x_test_centre = x_test_brut-np.tile(moy_train,(N_test,1))
y_test = np.array([1 for _ in range(411)] + [2 for _ in range(464)]).reshape(-1,1)
proj_test = np.c_[y_test,pca.transform(x_test_centre)[:, 0:2]]

In [ ]:
lda_classifier = LDA(proj_train, proj_test)
qda_classifier = QDA(proj_train,proj_test)

In [ ]:
lda_classifier.error_rate()

In [ ]:
qda_classifier.error_rate()

We get a very good error rate. In our case, the QDA performs better than the LDA, let's look at the confusion matrix: 

In [ ]:
preds = qda_classifier.classification(train=False)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
ConfusionMatrixDisplay.from_predictions(proj_test[:, 0], preds, labels = [1, 2])

We obtain a confusion matrix as good as in the previous tp by using only two components. In addition, we used much less data for training than in the previous tp which shows the robustness of the model. 

Nearest Neighbors : 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
n_neighbors = [i for i in range(2,20)]
n_best = -1
best_error_rate = 1.
for n in n_neighbors:
    knn_classifier = KNeighborsClassifier(n_neighbors = n)
    knn_classifier.fit(proj[:,0:2], y_train.reshape(1,-1)[0])
    error_rate = 1 - knn_classifier.score(proj_test[:, 1:3], y_test.reshape(1,-1)[0])
    if error_rate<best_error_rate:
        best_error_rate=error_rate
        n_best = n
        best_knn = knn_classifier
    

In [ ]:
print(f"The best value for n_neighbors is : {n_best}.", f"The error rate associated is {best_error_rate}")

In [ ]:
from sklearn.metrics import plot_confusion_matrix

In [ ]:
plot_confusion_matrix(best_knn, proj_test[:, 1:3], y_test.reshape(1,-1)[0])

PCA + KNN is better than PCA + QDA/LDA which could be predicted from the decision frontier using the first two components of the cpd. Indeed, it is impossible to construct a perfect quadratic or linear decision frontier from the data used during training.

We have seen that using PCA in advance of a classification is a very good way to retrieve only the important information from an image (works with any type of input) that allows the classification. To go further, we could do a Fisher dimension reduction coupled with one of the algorithms mentioned above.

## Beyong this lab

Have a look at other examples of applications, like

http://scikit-learn.org/stable/auto_examples/decomposition/plot_faces_decomposition.html#sphx-glr-auto-examples-decomposition-plot-faces-decomposition-py
    